In [1]:
%pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 7.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium

# Create a map centered on a specific location
map_center = [35.2828, -120.6596]  # Coordinates for San Luis Obispo, CA
m = folium.Map(location=map_center, zoom_start=13)

# Add a few markers
folium.Marker(location=[35.2828, -120.6596], popup='IPP', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(location=[35.2928, -120.6596], popup='Home', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=[35.2828, -120.6496], popup='Work', icon=folium.Icon(color='red')).add_to(m)

# Draw some lines (paths)
folium.PolyLine(locations=[[35.2828, -120.6596], [35.2928, -120.6596]], color='blue').add_to(m)
folium.PolyLine(locations=[[35.2828, -120.6596], [35.2828, -120.6496]], color='green').add_to(m)

# Add a circle representing a radius around a point
folium.Circle(location=[35.2828, -120.6596], radius=1000, color='purple', fill=True, fill_opacity=0.2).add_to(m)

# Save the map to an HTML file
m.save('dummy_map.html')

# Alternatively, if using a Jupyter notebook, display the map inline
m


In [4]:
%pip install requests shapely geopandas folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 7.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 16.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 16.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
import folium
from shapely.geometry import LineString
import geopandas as gpd

# Function to fetch street data from OSM using Overpass API
def fetch_street_data(lat, lon, radius):
    query = f"""
    [out:json];
    (
      way["residential"](around:{radius * 1609.34},{lat},{lon});
    );
    out body;
    >;
    out skel qt;
    """
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

# Function to analyze the fetched street data
def analyze_street_data(data):
    elements = data.get('elements', [])
    roads = []

    for element in elements:
        if element['type'] == 'way':
            coords = [(nd['lon'], nd['lat']) for nd in element.get('geometry', [])]
            if coords:
                road = LineString(coords)
                roads.append(road)

    gdf = gpd.GeoDataFrame(geometry=roads)
    return gdf

# Location parameters
lat = 35.2828  # Latitude of the point of interest
lon = -120.6596  # Longitude of the point of interest
radius = 1000  # Radius in miles

# Fetch and analyze the street data
try:
    street_data = fetch_street_data(lat, lon, radius)
    print("Street data fetched successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error fetching street data: {e}")
    street_data = {}

if street_data:
    street_gdf = analyze_street_data(street_data)
    print(f"Number of roads fetched: {len(street_gdf)}")

    # Create a map centered on the specified location
    m = folium.Map(location=[lat, lon], zoom_start=14)

    # Add the streets to the map
    for line in street_gdf.geometry:
        folium.PolyLine(locations=[(point[1], point[0]) for point in line.coords], color='blue').add_to(m)

    # Add a marker for the center point
    folium.Marker(location=[lat, lon], popup='Center Point', icon=folium.Icon(color='red')).add_to(m)

    # Save the map to an HTML file
    m.save('streets_map.html')

    # Alternatively, if using a Jupyter notebook, display the map inline
    m
else:
    print("No street data available.")


Street data fetched successfully.
Number of roads fetched: 0
